In [1]:
import json
import pandas as pd
import altair as alt
from pathlib import Path

In [2]:
data_dir = Path("../KIT-TIP-NLP")

In [3]:
systems = ["A_1", "A_2", "A_3", "A_4"]
languages = ["en", "es", "it"]
language_names = {"en": "English", "es": "Spanish", "it": "Italian"}

In [4]:
all_data = []

for lang in languages:
    for system in systems:
        json_file = data_dir / f"report_{lang}_{system}.json"
        
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            # Extract metrics for each class and macro avg
            for class_name, metrics in data.items():
                all_data.append({
                    "System": system,
                    "Language": language_names[lang],
                    "Class": class_name,
                    "Precision": metrics.get("precision"),
                    "Recall": metrics.get("recall"),
                    "F1-Score": metrics.get("f1-score")
                })
        except FileNotFoundError:
            print(f"File not found: {json_file}")

df_consolidated = pd.DataFrame(all_data)

In [5]:
print("=" * 100)
print("CONSOLIDATED METRICS TABLE - ALL SYSTEMS, LANGUAGES, AND CLASSES")
print("=" * 100)
print(df_consolidated.to_string(index=False))

CONSOLIDATED METRICS TABLE - ALL SYSTEMS, LANGUAGES, AND CLASSES
System Language     Class  Precision   Recall  F1-Score
   A_1  English         0   0.932476 0.463259  0.618997
   A_1  English         1   0.101604 0.644068  0.175520
   A_1  English macro avg   0.517040 0.553663  0.397258
   A_2  English         0   0.930348 0.597444  0.727626
   A_2  English         1   0.109541 0.525424  0.181287
   A_2  English macro avg   0.519944 0.561434  0.454457
   A_3  English         0   0.921569 0.675719  0.779724
   A_3  English         1   0.101770 0.389831  0.161404
   A_3  English macro avg   0.511669 0.532775  0.470564
   A_4  English         0   0.930348 0.597444  0.727626
   A_4  English         1   0.109541 0.525424  0.181287
   A_4  English macro avg   0.519944 0.561434  0.454457
   A_1  Spanish         0   0.950113 0.843058  0.893390
   A_1  Spanish         1   0.458333 0.750000  0.568966
   A_1  Spanish macro avg   0.704223 0.796529  0.731178
   A_2  Spanish         0   0.960591 0.

In [6]:
df_macro = df_consolidated[df_consolidated["Class"] == "macro avg"].copy()

print("\n" + "=" * 100)
print("MACRO AVERAGE METRICS - SUMMARY VIEW")
print("=" * 100)
macro_pivot = df_macro.pivot_table(
    index=["System", "Language"],
    values=["Precision", "Recall", "F1-Score"],
    aggfunc='first'
).round(4)
print(macro_pivot)


MACRO AVERAGE METRICS - SUMMARY VIEW
                 F1-Score  Precision  Recall
System Language                             
A_1    English     0.3973     0.5170  0.5537
       Italian     0.7659     0.7853  0.7509
       Spanish     0.7312     0.7042  0.7965
A_2    English     0.4545     0.5199  0.5614
       Italian     0.8103     0.8461  0.7853
       Spanish     0.7016     0.6814  0.8014
A_3    English     0.4706     0.5117  0.5328
       Italian     0.7659     0.7853  0.7509
       Spanish     0.7332     0.7461  0.7225
A_4    English     0.4545     0.5199  0.5614
       Italian     0.7523     0.7302  0.8013
       Spanish     0.6714     0.6611  0.7823


In [7]:
df_viz = df_macro.melt(
    id_vars=["System", "Language", "Class"],
    value_vars=["Precision", "Recall", "F1-Score"],
    var_name="Metric",
    value_name="Score"
)

In [8]:
chart_f1 = alt.Chart(df_macro).mark_bar().encode(
    x=alt.X("System:N", title="System"),
    y=alt.Y("F1-Score:Q", title="F1-Score", scale=alt.Scale(domain=[0, 1])),
    color=alt.Color("Language:N", title="Language"),
    column=alt.Column("Language:N", title="Language")
).properties(
    width=200,
    height=300,
    title="F1-Score Comparison Across Systems and Languages"
).interactive()
chart_f1.save('../figures/chart_f1.svg')


![](../figures/chart_f1.svg)

In [9]:
chart_all_metrics = alt.Chart(df_viz).mark_bar().encode(
    x=alt.X("System:N", title="System"),
    y=alt.Y("Score:Q", title="Score", scale=alt.Scale(domain=[0, 1])),
    color=alt.Color("Metric:N", title="Metric"),
    column=alt.Column("Language:N", title="Language"),
    row=alt.Row("Metric:N", title="Metric")
).properties(
    width=200,
    height=150
).interactive()

chart_all_metrics.save('../figures/chart_all_metrics.svg')

![](../figures/chart_all_metrics.svg)

In [10]:
chart_f1.show()

alt.Chart(...)

In [11]:
chart_all_metrics.show()

alt.Chart(...)